<a href="https://colab.research.google.com/github/AndreYang333/ExplainableAI/blob/main/Assignment9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 8
## Minjie Yang(my189)
Link to github:https://github.com/AndreYang333/ExplainableAI.git

In [ ]:
!pip install langchain vllm gptcache modelscope
!pip install transformers==4.32.0 accelerate tiktoken einops scipy transformers_stream_generator==0.0.4 peft deepspeed
!pip install auto-gptq optimum
!pip install --upgrade transformers

In [ ]:
from vllm import LLM, SamplingParams
import time
import os

#This is a Chinese LLM model
llm = LLM(model="qwen/Qwen-7B-Chat-int4", trust_remote_code=True,gpu_memory_utilization=0.55,max_model_len=1000)


In [56]:
prompts = ['''What are the benefits of studying explainability of AI?''']
sampling_params = SamplingParams(temperature=0.8,top_k=3, top_p=0.95,max_tokens=256,stop=["<|endoftext|>","<|im_end|>"])
start_time = time.time()
outputs = llm.generate(prompts, sampling_params)
end_time = time.time()
latency = end_time - start_time
print(f"Latency: {latency} seconds")
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt} \nGenerated text: \n{generated_text}")


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.05s/it, est. speed input: 10.50 toks/s, output: 41.99 toks/s]

Latency: 1.0592303276062012 seconds
Prompt: What are the benefits of studying explainability of AI? 
Generated text: 
 

  1. Improved trust in AI systems
  2. Better decision-making by humans
  3. Increased accountability for AI systems
  4. Improved understanding and control of AI systems.


## Perturbations of prompt

In [65]:
def perturb_prompt(prompt, perturb_type):
    perturbed_prompts = []
    if perturb_type == "symbol":
        # Add different symbols
        perturbed_prompts.append(prompt + "!")
        perturbed_prompts.append("?" + prompt)
    elif perturb_type == "pattern":
        # Change words
        perturbed_prompts.append(prompt.replace("benefits", "advantages"))
        perturbed_prompts.append(prompt.replace("AI", "Artificial Intelligence"))
    elif perturb_type == "text":
        # Change more words
        perturbed_prompts.append(prompt.replace("explainability of AI", "XAI"))
    return perturbed_prompts



In [59]:
# Perturbations of symbol
perturbed_symbol_prompts = perturb_prompt(prompts[0], 'symbol')
print(perturbed_symbol_prompts)
outputs = llm.generate(perturbed_symbol_prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt} \nGenerated text: \n{generated_text}")

['What are the benefits of studying explainability of AI?!', '?What are the benefits of studying explainability of AI?']


Processed prompts: 100%|██████████| 2/2 [00:06<00:00,  3.10s/it, est. speed input: 3.71 toks/s, output: 82.60 toks/s]

Prompt: What are the benefits of studying explainability of AI?! 
Generated text: 
 Explainability of AI is the ability of a machine learning model to provide clear and understandable insights into its decision-making process. Here are some benefits of studying explainability of AI:

  1. Improved Trust: When people understand how an AI model arrived at its decision, they are more likely to trust the model's output. This is particularly important in applications where decisions have significant consequences, such as medical diagnosis or financial lending.
  2. Increased Transparency: Explainability of AI helps to increase transparency in the decision-making process. This can help to identify potential biases or errors in the model and ensure that the decision-making process is fair and unbiased.
  3. Better Accountability: By providing clear explanations of how an AI model arrived at its decision, organizations can be held accountable for the model's outputs. This is particularly impor

1. **Exclamation Mark at the End**:  
   Adding an exclamation mark made the model’s response more detailed and formal, starting with a definition and listing benefits like "improved trust" and "increased transparency." This suggests the model interpreted the exclamation as a prompt for a thorough answer.

2. **Question Mark at the Beginning**:  
   Placing a question mark at the start led the model to generate related questions on AI explainability and ethics, rather than answering directly. This implies the model saw the prompt as open-ended, steering it toward thematic exploration rather than specific answers.

In [60]:
# Perturbations of words
perturbed_pattern_prompts = perturb_prompt(prompts[0], 'pattern')
print(perturbed_pattern_prompts)
outputs = llm.generate(perturbed_pattern_prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt} \nGenerated text: \n{generated_text}")

['What are the advantages of studying explainability of AI?', 'What are the benefits of studying explainability of Artificial Intelligence?']


Processed prompts: 100%|██████████| 2/2 [00:05<00:00,  2.91s/it, est. speed input: 3.95 toks/s, output: 75.30 toks/s]

Prompt: What are the advantages of studying explainability of AI? 
Generated text: 
 Explainability of AI can help us understand how AI models make decisions, which can be useful for a variety of reasons. Some of the advantages of studying explainability of AI include:

  * Improved trust: When people can see how AI models make decisions, they are more likely to trust them. This is important because AI models are often used to make decisions that have a significant impact on people's lives, such as decisions about medical treatments, financial investments, or criminal justice.
  * Increased accountability: Explainability of AI can help hold AI models and their creators accountable for their decisions. This is important because AI models are often complex and difficult to understand, which can make it difficult to determine who is responsible for their decisions.
  * Better decision-making: Understanding how AI models make decisions can help people make better decisions themselves. For 


1. **Prompt: "What are the advantages of studying explainability of AI?"**  

   By using "advantages" instead of "benefits," the model still recognizes the prompt's focus on the positive outcomes of explainability in AI but slightly shifts the language toward practical, real-world applications (e.g., “medical treatments,” “financial investments”). This suggests that the model interprets "advantages" with an emphasis on tangible, user-facing benefits and applications.

2. **Prompt: "What are the benefits of studying explainability of Artificial Intelligence?"**  
   Changing "AI" to "Artificial Intelligence" leads the model to adopt a more formal tone, introducing topics like "legal and ethical considerations" and the model's adaptability. This suggests that the expanded terminology might prompt the model to take a more comprehensive view, extending the focus from user benefits to regulatory and ethical considerations, as well as internal model performance enhancements.

In [66]:
# Perturbations of texts
perturbed_texts_prompts = perturb_prompt(prompts[0], 'text')
print(perturbed_texts_prompts)
outputs = llm.generate(perturbed_texts_prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt} \nGenerated text: \n{generated_text}")

['What are the benefits of studying XAI?']


Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.77s/it, est. speed input: 1.56 toks/s, output: 44.37 toks/s]

Prompt: What are the benefits of studying XAI? 
Generated text: 
 

XAI has several benefits for both businesses and individuals. 

For businesses, XAI can help improve decision-making processes by providing insights into how decisions are made. This can help businesses make more informed decisions, reduce the risk of errors, and improve overall efficiency. XAI can also help businesses comply with regulations and ethical standards by providing transparent decision-making processes. 

For individuals, XAI can help improve transparency and accountability in decision-making processes. This can help individuals make more informed decisions and reduce the risk of errors or biases. XAI can also help individuals understand the reasoning behind decisions, which can improve their trust in decision-makers. 

In summary, XAI has the potential to improve decision-making processes, reduce the risk of errors, increase transparency and accountability, and improve trust in decision-makers. These benef

**Prompt**: `"What are the benefits of studying XAI?"`
  
   By using "XAI," the model still interprets the prompt as referring to "explainable AI" but leans towards a more technical and business-oriented perspective. The use of the acronym appears to signal a domain-specific context, prompting the model to focus on practical applications, compliance, and regulatory benefits for businesses. The response's structure—discussing benefits, then challenges—suggests the model is adopting a perspective often found in professional or academic discourse on XAI.


## Counterfactuals

In [73]:
counterfactual_prompts = [
    "What are the challenges of studying explainability of AI?",       # Replaces "benefits" with "challenges"           # Changes question to a declarative sentence
]

outputs = llm.generate(counterfactual_prompts, sampling_params)
# Print the outputs.
for output in outputs:
  prompt = output.prompt
  generated_text = output.outputs[0].text
  print(f"Prompt: {prompt} \nGenerated text: \n{generated_text}")

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.83s/it, est. speed input: 1.89 toks/s, output: 43.93 toks/s]

Prompt: What are the challenges of studying explainability of AI? 
Generated text: 
 How can these challenges be addressed?

AssistantStudying explainability of AI involves understanding how AI systems make decisions and how these decisions can be explained to humans. This can be challenging for several reasons:

1. Complexity of AI models: AI models are often complex and difficult to understand, even for experts in the field. This makes it challenging to explain how these models work and how they make decisions.

2. Lack of transparency: Many AI models are designed to be black boxes, meaning that their decision-making process is not transparent to humans. This can make it difficult to understand how the model arrived at a particular decision.

3. Ethical concerns: There are ethical concerns related to AI explainability, such as the potential for AI models to discriminate against certain groups of people or to be used for malicious purposes.

4. Data privacy: AI models often rely on la


**Analysis**:  
   By asking about challenges instead of benefits, the model shifts its response to focus on inherent difficulties in making AI explainable. This altered prompt encouraged the model to provide a structured view of the barriers to explainability, highlighting complex technical and ethical dimensions. The inclusion of potential solutions suggests the model is programmed to offer a balanced perspective, addressing both problems and their possible remedies when faced with a challenge-oriented prompt.

In [74]:
counterfactual_prompts = [
    "What are the benefits of studying transparency of AI?",           # Replaces "explainability" with "transparency",       # Replaces "benefits" with "challenges"           # Changes question to a declarative sentence
]

outputs = llm.generate(counterfactual_prompts, sampling_params)
# Print the outputs.
for output in outputs:
  prompt = output.prompt
  generated_text = output.outputs[0].text
  print(f"Prompt: {prompt} \nGenerated text: \n{generated_text}")

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.81s/it, est. speed input: 1.72 toks/s, output: 44.07 toks/s]

Prompt: What are the benefits of studying transparency of AI? 
Generated text: 
 What are some potential risks and challenges associated with transparency of AI? How can we ensure that AI is developed and used in a transparent manner? What role do government and industry regulations play in promoting transparency of AI? How can we ensure that AI is used for the benefit of society as a whole, and not just for the benefit of a select few? How can we address the concerns around bias and discrimination in AI? How can we ensure that AI is used ethically and responsibly?

Transparency of AI is important because it allows us to understand how AI systems work and make informed decisions about their use. It also allows us to identify and correct any biases or errors in the AI system, which is crucial for ensuring that AI is used fairly and justly. Additionally, transparency of AI can help build trust in AI systems, which is essential for their adoption and widespread use.

There are several pot

**Analysis**:  
   The use of “transparency” rather than “explainability” in the prompt seems to have led the model to emphasize not only the benefits but also the broader implications, including risks and societal challenges. Transparency as a concept often implies both visibility and accountability, prompting the model to consider ethical, regulatory, and societal aspects. This approach reflects a more balanced view, integrating the benefits of transparency with a cautious awareness of its limitations and challenges.

## Shap

In [ ]:
!pip install shap

In [82]:
import numpy as np

# Original input text
original_prompt = "What are the benefits of studying explainability of AI?"

# Tokenize the text into a list of words
tokens = original_prompt.split()

# Get the model output for the original prompt
original_output = llm.generate([original_prompt], sampling_params)
original_text = original_output[0].outputs[0].text
original_score = len(original_text)  # Using generated text length as an example score

# Placeholder list to store saliency scores
saliency_scores = []

# Mask each word and calculate the saliency score
for i in range(len(tokens)):
    # Mask the i-th word to create a counterfactual prompt
    masked_prompt = " ".join(tokens[:i] + tokens[i+1:])
    masked_output = llm.generate([masked_prompt], sampling_params)
    masked_text = masked_output[0].outputs[0].text
    masked_score = len(masked_text)  # Using generated text length as a score

    # Calculate the saliency score as the difference between the original and masked scores
    saliency_score = abs(original_score - masked_score)
    saliency_scores.append(saliency_score)

# Output each word and its corresponding saliency score
for token, score in zip(tokens, saliency_scores):
    print(f"Token: {token}, Saliency Score: {score}")


Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.62s/it, est. speed input: 1.95 toks/s, output: 44.21 toks/s]

Token: What, Saliency Score: 61
Token: are, Saliency Score: 1420
Token: the, Saliency Score: 1420
Token: benefits, Saliency Score: 1420
Token: of, Saliency Score: 1420
Token: studying, Saliency Score: 809
Token: explainability, Saliency Score: 1420
Token: of, Saliency Score: 486
Token: AI?, Saliency Score: 211


SHAP analysis, based on output length, highlights the importance of each token in shaping the response:

1. **High Saliency**: Tokens like **"are," "the," "benefits," "of,"** and **"explainability"** have high scores (1420), showing they are crucial in guiding the model’s response content.

2. **Moderate Saliency**: **"studying"** (809) and the second **"of"** (486) add context but are less influential than the main terms.

3. **Low Saliency**: **"What"** (61) and **"AI?"** (211) have minimal impact, serving more as structural elements.

This suggests the model relies most on specific, content-rich terms to generate a detailed response, with introductory or generic terms contributing less.

## Summary

These findings highlight the model's responsiveness to small prompt changes—punctuation, synonyms, acronyms, and thematic shifts. Even minor adjustments influence how the model interprets context, affecting response style, tone, and focus. This adaptability shows the importance of prompt engineering for tailored outputs.

### Interesting Findings

1. **Punctuation Sensitivity**: A question mark at the start made the prompt open-ended, while an exclamation mark led to a more formal, detailed response, indicating the model's sensitivity to punctuation cues.

2. **Synonym Choice**: Using "advantages" instead of "benefits" and "transparency" instead of "explainability" shifted the focus, with "transparency" prompting more ethical discussion, showing the model's nuanced understanding of word choice.

3. **Acronym Influence**: "XAI" led to a technical, business-oriented response, suggesting the model treats acronyms as cues for a specialized context.

4. **Saliency Scores**: Core terms like "benefits" and "explainability" had high saliency, heavily influencing response content, while structural words like "What" had minimal impact, showing the model's reliance on content-rich words.

5. **Balanced Responses in Open Prompts**: When asked about "challenges" or "transparency," the model provided risks and solutions, demonstrating its ability to consider broader perspectives in nuanced prompts.